<a href="https://colab.research.google.com/github/KevinTheRainmaker/ML_DL_Basics/blob/master/HonGong_ML_DL/16_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 심층 신경망

### **키워드:** 심층 신경망, ReLU 함수, 옵티마이저

케라스 인공 신경망에 은닉층을 추가하여 심층 신경망을 만들고 분류 과제를 수행해보자.

In [1]:
# packages
from tensorflow import keras

from sklearn.model_selection import train_test_split

## 은닉층

은닉층(hidden layer)란 입력층과 출력층 사이 존재하는 모든 층을 지칭하는 용어이다. 은닉층에 적용되는 활성화 함수는 출력층과 달리 다소 자유로운 편으로, 시그모이드 혹은 렐루(ReLU) 함수가 주로 이용된다.

은닉층에서 활성화 함수를 사용함으로써 선형 계산을 비선형 계산으로 틀어 계산의 복잡도와 정확도를 높인다.

In [2]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)) # 은닉층
dense2 = keras.layers.Dense(10, activation='softmax') # 출력층

`dense1`은 은닉층으로 100개의 뉴런을 가진 밀집층이다. 활성화 함수로 sigmoid를 지정했으며 `input_shape` 매개변수에서 입력의 크기를 (784,)로 지정했다.

은닉층의 뉴런 개수를 정하는 기준은 출쳑층의 뉴런보다 많아야 한다는 점 외에는 특별히 존재하지 않으며, 경험적으로 정해지곤 한다.

`dense1`는 출력층으로, 10개의 뉴런을 가지고 클래스를 분류한다. 다중 확률 값 중 최대치를 출력하기 위해 활성화 함수는 softmax로 설정하였다.

## 심층 신경망 만들기